# Causal Inference for Credit Risk Analysis

### Project Objective
This notebook explores the causal drivers of loan default among post-resignation accounts. While traditional predictive models can identify *who* is likely to default, this analysis aims to understand *why* they default. The primary goal is to use these insights to inform a more effective, proactive risk mitigation strategy.

## 1. Data Ingestion and Preparation

The first step is to load and merge the necessary datasets: the loan-level historical data and the user-level data containing employment and default information. In the original environment, this data was queried from a Databricks data lake using Spark SQL.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# --- SANITIZED DATA LOADING ---
# In the original environment, this data was loaded directly from production tables.
# For this portfolio, we are simulating the data loading step.

# SANITIZED: Replaced Spark SQL queries with a comment explaining the source.
# df_user = spark.sql("SELECT * FROM `[PROD_DATABASE]`.`[USER_SALARY_DEFAULT_TABLE]`").toPandas()
# df_loan = spark.sql("SELECT * FROM `[PROD_DATABASE]`.`[LOAN_LEVEL_DATASET]`").toPandas()

# For demonstration purposes, we will assume df_loan and df_user are pre-loaded
# as pandas DataFrames with the correct schemas.

# Perform the merge as in the original script
# user_cols_to_merge = [...] 
# df_user_subset = df_user[user_cols_to_merge].copy()
# ... (rest of the merging logic would go here) ...

## 2. Feature Engineering & Exploratory Analysis

With the data merged, the next step is to engineer the key feature for this analysis: the **Debt-to-Income (DTI) ratio**. We then perform an initial exploratory analysis to observe the correlation between DTI and the historical default rate.

In [ ]:
# Create a working copy of the dataframe
# df_resigned = df[df['user_resigned_at'].notnull()].copy()

# --- FEATURE ENGINEERING: Debt-to-Income (DTI) ---
# df_resigned['dti'] = df_resigned['initial_unpaid_amount_user_level'] / df_resigned['base_monthly_salary_user_level']

# Create DTI bins for easier visualization and analysis
# dti_bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 1]
# dti_labels = ['<=10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '>70%']
# df_resigned['dti_bin'] = pd.cut(df_resigned['dti'], bins=dti_bins, labels=dti_labels)

# --- EXPLORATORY ANALYSIS ---
# df_pivot = pd.pivot_table(df_resigned, index='dti_bin', columns='user_level_default_tag',
#                values='user_reference_code', aggfunc='count')
# df_pivot['default_rate'] = df_pivot[True] / (df_pivot[True] + df_pivot[False])

# plt.figure(figsize=[10,5])
# plt.bar(df_pivot.index, df_pivot['default_rate'])
# plt.title('Observed Default Rate by DTI Ratio');
# plt.show()

## 3. Causal Analysis: Identifying Key Drivers

This is the core of the analysis. Instead of just observing the correlation, we now apply causal inference techniques to understand if a high DTI ratio is a **true cause** of default. This involves creating treatment and control groups and estimating the causal impact.

For this analysis, we define our treatment group as users with a DTI ratio **greater than 50%**.

In [ ]:
# --- CAUSAL ANALYSIS SETUP ---

# In a real scenario, this is where you would import and use a causal inference library
# like DoWhy, EconML, or a custom implementation.

# 1. Define the Treatment, Outcome, and Confounders
# treatment = 'dti_greater_than_50_percent' # Binary variable (True/False)
# outcome = 'user_level_default_tag' # Binary variable (True/False)
# confounders = ['employment_tenure_months', 'job_class', 'base_monthly_salary'] # Example confounders

# 2. Create a Causal Model
# model = CausalModel(
#     data=df_resigned,
#     treatment=treatment,
#     outcome=outcome,
#     common_causes=confounders
# )

# 3. Estimate the Causal Effect
# identified_estimand = model.identify_effect()
# estimate = model.estimate_effect(identified_estimand,
#                                method_name="backdoor.propensity_score_weighting")

# print("Causal Estimate is ##", estimate)
# print(estimate)

## 4. Policy Simulation & Impact Assessment

Based on the findings from the causal analysis, the final step is to simulate the impact of a new business policy. The analysis revealed that a DTI ratio above a certain threshold was a significant causal driver of default.

The proposed policy was to **adjust the credit limit for certain customer segments to keep their DTI below 50%**. This section of the code simulates the financial impact of this proposed change.

In [ ]:
# --- POLICY SIMULATION ---

# Load the active user credit profile data
# SANITIZED: Replaced Spark SQL query
# df_policy_sim = spark.sql("SELECT * FROM `[PROD_DATABASE]`.`[VERIFIED_USERS_CREDIT_PROFILE]`").toPandas()

# Filter for the relevant, active user base
# (black_list and other filters would be applied here)

# Calculate the proposed adjusted credit limit
# df_policy_sim['credit_limit_pct'] = df_policy_sim['credit_limit'] / df_policy_sim['salary']
# df_policy_sim['credit_limit_adjusted'] = df_policy_sim['salary'] * 0.5 # New 50% DTI ceiling
# df_policy_sim['credit_limit_adjusted'] = np.floor(df_policy_sim['credit_limit_adjusted'] / 100) * 100

# Calculate the total adjustment in credit limits
# adjustment_impact = df_policy_sim[df_policy_sim['credit_limit_pct'] > 0.5][['credit_limit', 'credit_limit_adjusted']].sum()
# total_reduction = adjustment_impact.diff()['credit_limit_adjusted']

# print(f"Total projected reduction in credit limit exposure: {total_reduction:,.2f}")